```{currentmodule} optimap
```

In [ ]:
from optimap.utils import jupyter_render_animation as render

```{tip}
Download this tutorial as a {download}`Jupyter notebook <converted/overview.ipynb>`, or a {download}`python script <converted/overview.py>` with code cells. We highly recommend using [Visual Studio Code](#vscode) to execute this tutorial. Alternatively, you could run the Python script in a terminal with ``python overview.py`` from the folder where the file is located.
```

# Tutorial 0: Overview of optimap

Welcome to optimap! This tutorial will give you a hands-on introduction to the main features of optimap, a Python library designed for analyzing fluorescent high-dynamic range video data. While our examples focus on cardiac optical mapping, optimap is versatile and can be used for many types of fluorescent imaging applications, including calcium imaging in neurons, fluorescent imaging in cell cultures, and other dynamic biological systems.

**What is optical mapping?** Optical mapping is a technique used to record biological activity using fluorescent indicators. In cardiac research, these dyes change their fluorescence in response to changes in voltage or calcium, allowing researchers to visualize electrical waves spreading across the heart. Similar principles apply to other fluorescent imaging techniques in various biological systems.

**What can optimap do?** Optimap helps you:
- Load and visualize fluorescent video data from various sources and formats
- Extract and analyze signals from specific regions of interest
- Correct for tissue motion to improve signal quality (essential for many living samples)
- Process and enhance signals to better visualize dynamic wave phenomena
- Analyze spatiotemporal patterns in fluorescence data

In this tutorial, we'll use cardiac data as an example, but the techniques demonstrated can be applied to other types of fluorescent imaging data. Let's get started by importing the optimap library. We'll use the short name `om` to make our code more concise:

In [ ]:
# Import optimap and use the short name 'om'
import optimap as om

If there's no error message after running this cell, optimap has been imported successfully! If you encounter an error, please check that optimap is properly installed (see [Installation Guide](#installing)).

We'll also import two other common libraries that we will use in this tutorial:

In [ ]:
# NumPy helps us work with numerical data arrays
import numpy as np

# Matplotlib helps us create plots and visualizations
import matplotlib.pyplot as plt

## 1. Loading Video Data

Optimap can read many different video file formats used in optical mapping research, see [](io.ipynb) for further details.

:::{admonition} File formats supported by {func}`optimap.load_video`
:class: tip, dropdown

* .tif, .tiff (TIFF) image stacks
* Folder containing sequence of TIFF or .png (PNG) images
* .mp4, .avi, .mov, … (digital video files)
* .mat (MATLAB)
* .npy (numpy array)
* .gsd, .gsh (SciMedia MiCAM 05)
* .rsh, .rsm, .rsd (SciMedia MiCAM ULTIMA)
* .dat (MultiRecorder)
:::

Let's download an example video file and load it:

In [ ]:
# Download an example file (only needs to be done once)
filename = om.download_example_data("VF_Rabbit_1.npy")

# Load the video file
# (If using your own data, replace the filename with your file path)
video = om.load_video(filename)

# Print information about the video
om.print_properties(video)

The output above tells us important information about our video:

- **Shape**: The video has 1000 frames, with each frame being 390×300 pixels
- **Data type**: The pixel values are stored as 16-bit unsigned integers (uint16)
- **Value range**: The pixel values range from 51 to 3884

In optimap, videos are stored as 3D numpy arrays where the first dimension represents time (frames), and the second and third dimensions represent the height and width of each frame, respectively.

### About the Example Video

This example recording shows a ex-vivo isolated rabbit heart during ventricular fibrillation. THe heart was stained with a voltage-sensitive fluorescent dye (Di-4-ANEPPS) and recorded at 500 frames per second. The heart is contracting slightly, which causes motion in the video. The data is from {cite:t}`Chowdhary2023`.

## 2. Playing Videos

Let's look at our video data using optimap's built-in viewer:

In [ ]:
# Display the video with a title
# skip_frame=3 means we only show every third frame (for speed)
om.show_video(video, title='Heart during Fibrillation', skip_frame=3)

In [ ]:
render(lambda: om.show_video(video, title='Heart during Fibrillation', interval=25),
       mp4_filename="VF_Rabbit_1_gui.mp4",
       save_args={"hide_slider": False, "hide_buttons": False, "hide_framecounter": True})

In the video above, you should observe the heart tissue with subtle variations in brightness (fluorescence intensity) that represent electrical activity. These changes correspond to action potentials propagating across the cardiac tissue during ventricular fibrillation.

You may also notice the slight motion of the heart tissue as it contracts. This movement creates what we call "motion artifacts" in optical mapping data, which can obscure the true electrical signals we're interested in.

The complex wave patterns (action potential wavefronts) are somewhat difficult to see in this raw footage. In the video below, they were enhanced and are shown in purple. In the following sections, we'll demonstrate techniques to enhance these signals, compensate for the motion, and extract meaningful data from this optical mapping recording. 

In [ ]:
filepath = om.utils.download_example_data('VF_Rabbit_1_warped.npy', silent=True)
video_warped = om.load_video(filepath, use_mmap=True)
# video_warped_normalized = om.video.normalize_pixelwise_slidingwindow(video_warped, window_size=60)
# signal = om.motion.reverse_motion_compensate(
#     video, video_warped_normalized,
#     contrast_kernel=5,
#     presmooth_spatial=1,
#     presmooth_temporal=1
# )
# om.save_video('VF_Rabbit_1_rwarped_normalized_f16.npy', r.astype(np.float16))
filepath = om.utils.download_example_data('VF_Rabbit_1_rwarped_normalized_f16.npy', silent=True)
signal = om.load_video(filepath)
mask = om.background_mask(video_warped[0], show=False)
signal[:, mask] = np.nan
alpha = om.video.normalize(signal, vmin=0.5, vmax=0)


In [ ]:
render(lambda: om.video.show_video_overlay(video, overlay=(1-signal), alpha=alpha, vmax_overlay=1, interval=25))

### Alternative Viewer: Monochrome

[Monochrome](https://github.com/sitic/monochrome) is a companion software for visualizing monochromatic video data. In certain aspects, yt is more powerful and flexible than the built-in viewer functions in optimap ({func}`show_video`, {func}`show_video_overlay`, ...). You can use Monochrome to visualize the same video data we just loaded.

In [ ]:
# This will open the video in Monochrome in a new window
import monochrome as mc
mc.show(video, name="Heart Recording")

[<center><img src="https://cardiacvision.github.io/optimap/main/_static/Monochrome-screenshot1.webp"></center>](https://github.com/sitic/monochrome/)

Monochrome allows you to click on the video to see the signal traces at specific locations, adjust brightness/contrast, and more.

## 3. Extracting and Analyzing Signals

In optical mapping, we're often interested in the changes in fluorescence over time at specific locations on the heart. These time-series are called "traces" and represent the electrical activity.

### Selecting Traces Interactively

Let's select some points on the heart and look at their signals:

In [ ]:
# This opens an interactive window where you can click to select positions
# fps=500 tells optimap the video was recorded at 500 frames per second
traces, positions = om.select_traces(video, size=5, fps=500)

In [ ]:
positions = [(127, 147), (130, 209), (202, 136)]

fig, axs = plt.subplots(1, 2, figsize=(11,4))
om.trace.show_positions(positions, video[0], ax=axs[0])
traces = om.trace.extract_traces(video, positions, size=5, ax=axs[1], show=True, fps=500)
plt.show()

In the interactive mode, you can:
- **Left-click** on the image to add points
- **Right-click** on a point to remove it
- **Close** the window when you're done

### Understanding Trace Extraction

The `extract_traces` function pulls signals from specific locations in the video. Instead of using a single pixel (which can be noisy), optimap averages over a small window around each position:

In [ ]:
# Extract traces from our previously selected positions
# size=1 means we only use the exact pixel (no averaging)
traces = om.extract_traces(video, positions, size=1, show=True, fps=500)

Notice how the traces with `size=1` look more noisy compared to the previous traces with `size=5`.

You can customize how the signals are extracted and displayed:

**Notice the problem:** The traces show large fluctuations that make it hard to see the actual cardiac signals. These are **motion artifacts** caused by the heart tissue moving during recording. In the next section, we'll fix this problem!

## 4. Compensating for Motion Artifacts

Motion is a major challenge in optical mapping. When the heart contracts, the tissue moves under the camera, causing changes in the optical trace that aren't related to electrical activity. 

Optimap can track and correct for this motion using a process called "motion compensation". Tracking and stabilizing motion in fluorescence videos requires specialized algorithms, see [](motion_compensation) and {footcite:t}`Christoph2018a, Lebert2022` for details.

In [ ]:
# Track and correct for motion in the video
# This may take a few minutes depending on your computer
video_warped = om.motion_compensate(video,
                                    contrast_kernel=5,      # Size of contrast enhancement kernel
                                    presmooth_spatial=1,    # Amount of spatial smoothing
                                    presmooth_temporal=1)   # Amount of temporal smoothing

In [ ]:
# load pre-computed warped video to speed up docs build time
filepath = om.utils.download_example_data('VF_Rabbit_1_warped.npy', silent=True)
video_warped = om.load_video(filepath, use_mmap=True)

In brief, the function works by:
1. Tracking the motion of each pixel between frames
2. Warping each frame to align with a reference frame
3. Creating a new video where the tissue appears stationary

Let's compare the original and motion-corrected videos side by side:

In [ ]:
# Show original and motion-corrected videos side-by-side
om.show_video_pair(video,
                   video_warped,
                   title1="Original (with motion)",
                   title2="Corrected (stabilized)",
                   skip_frame=3)

In [ ]:
render(lambda: om.video.play2(video, video_warped, title1="Original (with motion)", title2="Corrected (stabilized)", skip_frame=1))

The differences may be subtle, but in the right video (corrected), the heart tissue stays more stationary.

### Examining Traces After Motion Correction

Now let's see how our traces look with motion correction:

In [ ]:
# Show traces from the same positions in the motion-corrected video

fig, axs = plt.subplots(1, 2, figsize=(11,4))
om.trace.show_positions(positions, video[0], ax=axs[0])
traces_corrected = om.trace.extract_traces(video_warped, positions, size=5, ax=axs[1], fps=500)
plt.show()

**Big improvement!** Notice how the traces now show clearer patterns with much less noise. The baseline fluctuations have been significantly reduced, making it easier to see the actual electrical signals.

## 5. Saving and Exporting Results

Once you've processed your data, you might want to save it for later analysis or to share with others:

In [ ]:
# Save the motion-corrected video as a TIFF stack
# This preserves all the data for future analysis
om.video.save_video('motion_corrected_heart.tiff', video_warped)

# Export as a video file that can be played in standard media players
# fps=50 sets the playback speed
om.video.export_video('motion_corrected_heart.mp4', video_warped, fps=50)

## 6. Enhancing Cardiac Wave Visualization

The electrical signals in optical mapping are often small compared to the background fluorescence. We can enhance them to make the cardiac waves more visible using normalization techniques. See [](signal_extraction) for more details.

### Sliding-Window Normalization

This technique adjusts each pixel's intensity based on its local minimum and maximum over time, making the waves stand out:

In [ ]:
# Apply sliding-window normalization to enhance wave visualization
# window_size=60 means we use 60 frames (120ms at 500fps) for each calculation
video_warped_normalized = om.video.normalize_pixelwise_slidingwindow(video_warped, window_size=60)

### Creating a Mask for the Background

To focus only on the heart tissue, we'll create a mask that identifies the background:

In [ ]:
# Create a mask that identifies background pixels
mask = om.background_mask(video_warped[0])

# Set all background pixels to NaN in our normalized video
video_warped_normalized[:, mask] = np.nan

[](masks) provides on overview of several methods in optimap to create masks, from automatic detection to manual drawing.

Now let's look at our pixelwise normalized video, where the action potential waves appear as dark waves moving across the heart tissue:

In [ ]:
om.video.play(video_warped_normalized, interval=20)

In [ ]:
render(lambda: om.video.play(video_warped_normalized, interval=40))

## Enhanced Wave Visualization and Overlays

In the video above, you can see the normalized cardiac electrical activity clearly. The normalization technique has helped to:

1. **Amplify small changes** in fluorescence that represent electrical activity
2. **Remove background variation** across the heart tissue
3. **Highlight the wave dynamics** of electrical propagation during fibrillation

The dark waves you observe represent action potentials (depolarization) moving across the tissue in complex patterns characteristic of fibrillation.

### Creating Video Overlays

We can further enhance visualization by creating **overlays** that combine the original video with the processed data. This allows us to see both the anatomical structure (from the original video) and the electrical activity (from the normalized video) simultaneously.

In the next cells, we'll create an overlay where:
- The background shows the original heart tissue
- Colored overlay shows the electrical wave activity
- Transparency (alpha) is controlled by the intensity of the normalized signal

In [ ]:
alpha = om.video.normalize(video_warped_normalized, vmin=0.5, vmax=0)
om.video.show_video_overlay(video_warped, overlay=(1-video_warped_normalized), alpha=alpha, interval=30)

In [ ]:
alpha = om.video.normalize(video_warped_normalized, vmin=0.5, vmax=0)
render(lambda: om.video.show_video_overlay(video_warped, (1-video_warped_normalized), alpha=alpha, interval=20))

### Next Steps

- Try loading your own optical mapping data
- Learn about activation maps and conduction velocity in later tutorials


```{footbibliography}
```